# Song Popularity Prediction Using Machine Learning

### Importing Data and Libraries

In [ ]:
# Import Statements
import numpy as np # Linear algebra and pandas compatibility
import pandas as pd # Data management, and dataframes
from sklearn.model_selection import train_test_split # Splits dataset into training|testing sets
from sklearn.tree import DecisionTreeRegressor # Decision Tree Model
from sklearn.ensemble import RandomForestRegressor # Random Forest Model
from sklearn.metrics import mean_absolute_error # MAE, measuring loss
import matplotlib.pyplot as plt # Graphing library to visualize the data/correlations
import seaborn as sns #Heatmap
from yellowbrick.model_selection import FeatureImportances # Correlation finding
from sklearn.linear_model import LogisticRegression
from yellowbrick.datasets import load_energy
from yellowbrick.model_selection import ValidationCurve

In [ ]:
# Importing our dataset: "Spotify dataset 1922-2021"
file_path = '../input/spotify-dataset-19212020-160k-tracks/data_o.csv' # CSV file
df = pd.read_csv(file_path) # Creating our main dataframe named "df" using pd.read_csv

### Visualizing Our Data

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

#### Looking for Obvious Trends

In [ ]:
Benchmark = df[['artists','year','name', 'release_date', 'popularity']]
# The Benchmark is the "Popularity" index, as that is our label
Benchmark = Benchmark.sort_values(by=['popularity', 'name'], ascending=[False, True])
Benchmark.artists = Benchmark.artists.str.strip('[]').str.replace("'", "")
Benchmark.head(50)

In [ ]:
# In the previous, it seems as though 2020 (newest) is favored. Is the opposite true for the oldest?
year = Benchmark.sort_values(by=['year', 'popularity'], ascending=[True, False])
year.head(10)

In [ ]:
# Perhaps this is biased, as the music from 1921 might actually be less popular. Let's use qualitative data to see if our hypothesis of new year = +popularity is true.
popularoldyear = df.loc[(df['year'] >= 1950) & (df['year'] <= 2000)]
popularoldyear = popularoldyear.sort_values(by=['popularity', 'name'], ascending=[False, True])
popularoldyear.head(20)

#### Now that we have a better idea as to how Spotify ranks the popularity of their music, we can start analyzing its correlation with other features on the Spotify API dataset.

In [ ]:
# Creating a simple heatmap to see the correlation of our features to our labels
sns.heatmap(df.corr(), cmap='icefire');
# With this heatmap, we can decide which values are most valuable (usually the close to 1.0, the better)
# We can also decide which ones we might want to make synthetic features with.

Looking at this graph, there are a couple pairs which have great correlation:
- Year - Popularity (Positive)
- Loudness - Energy (Positive)
- Energy - Acousticness (Negative)
- Acousticness - Year (Negative)

##### The only ones of these pairs that could give us meaningful synthetic data are Loudness - Energy, and Energy - Acousticness as the first includes the label, and the last includes our main feature, "year".

Also, mode and key have very little correlation with any values, we can remove them.


In [ ]:
df.corr()['popularity'].sort_values(ascending=False) # Numerical values to prove hypotheses

In [ ]:
df.corr()['energy'].sort_values(ascending=False)

In [ ]:
# Analyzing our main feature
y = df.popularity
 
fig,axs = plt.subplots(2,1, figsize=(7,7))
fig.suptitle('Observations of popularity')
 
# Observe the distribution of 'popularity'axs[0].set_title('Distribution of popularity')
axs[0].set_title('Distribution of popularity')
sns.distplot(df['popularity'], ax=axs[0], kde=False)
 
axs[1].set_title('Relationship between popularity and year')
sns.lineplot(x='year', y='popularity', data=df, ax=axs[1])
 
fig.tight_layout(pad=3.0)

In [ ]:
# Set the predictor variables
df["loud_energy"] = df["energy"] * df["loudness"]
df["acoustic_energy"] = df['acousticness'] * df['energy']
features = ['valence', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'explicit', 'year', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'loud_energy', 'acoustic_energy']
# Removed 'mode' and 'key', as they had little/no correllation with popularity.
X = df[features]
X.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size= 0.2, random_state=0)

### Training, Testing, and Predicting

In [ ]:
model = RandomForestRegressor()
viz = FeatureImportances(model)
viz.fit(train_X, train_y)
viz.show()
# Another correlation analysis, now using Machine Learning scripts to get a better understadning.

#### Comparing Models

In [ ]:
lr_model = LogisticRegression(random_state=0)
lr_model.fit(train_X,train_y)
val_preds1 = lr_model.predict(test_X)
val_mae1 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae1:.3f}')

In [ ]:
dec_tree = DecisionTreeRegressor(random_state=0)
dec_tree.fit(train_X, train_y)
val_preds1 = dec_tree.predict(test_X)
val_mae2 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae2:.3f}')
# print(f'Training Coefficient of R^2 : {dec_tree.score(train_X, train_y):.2f}')
# print(f'Test Coefficient of R^2 : {dec_tree.score(test_X, test_y):.2f}')

In [ ]:
rf_model = RandomForestRegressor(random_state=0)
rf_model.fit(train_X, train_y)
val_preds1 = rf_model.predict(test_X)
val_mae3 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae3:.3f}')
# Unfortunately, due to hardware limitaions, I couldn't include a validation curve to check for overfitting.

### Addressing a large problem in the dataset: The Year Bias

In [ ]:
df["loud_energy"] = df["energy"] * df["loudness"]
df["acoustic_energy"] = df['acousticness'] * df['energy']
features = ['valence', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'explicit', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'loud_energy', 'acoustic_energy']
# REMOVED YEAR
X = df[features]
X.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size= 0.2, random_state=0)

In [ ]:
lr_model = LogisticRegression(random_state=0)
lr_model.fit(train_X,train_y)
val_preds1 = lr_model.predict(test_X)
val_mae4 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae4:.3f}')

In [ ]:
train_X.head()

In [ ]:
dec_tree = DecisionTreeRegressor(random_state=0)
dec_tree.fit(train_X, train_y)
val_preds1 = dec_tree.predict(test_X)
val_mae5 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae5:.3f}')

In [ ]:
rf_model = RandomForestRegressor(random_state=0)
rf_model.fit(train_X, train_y)
val_preds1 = rf_model.predict(test_X)
val_mae6 = mean_absolute_error(test_y, val_preds1)
print(f'Mean absolute error of this model: {val_mae6:.3f}')
# Unfortunately, due to hardware limitaions, I couldn't include a validation curve to check for overfitting.

In [ ]:
data = [[ 31.641, 9.281,  6.753]]

columns = ('LogisticRegression', 'DecisionTree', 'RandomForest')
rows = ['%d Mean Absolute Error' % x for x in (100, 50, 20, 10, 5)]

values = np.arange(0, 2500, 500)
value_increment = 1000

# Get some pastel shades for the colors
colors = plt.cm.magma(np.linspace(0, 0.5, len(rows)))
n_rows = len(data)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.2, bottom=0.2)

plt.ylabel("Loss in ${0}'s".format(value_increment))
plt.yticks(values * value_increment, ['%d' % val for val in values])
plt.xticks([])
plt.title('Loss by Disaster')

plt.show()

In [ ]:
data = [[ 31.634, 14.183, 10.441]]

columns = ('LogisticRegression', 'DecisionTree', 'RandomForest')
rows = ['%d Mean Absolute Error' % x for x in (100, 50, 20, 10, 5)]

values = np.arange(0, 2500, 500)
value_increment = 1000

# Get some pastel shades for the colors
colors = plt.cm.magma(np.linspace(0, 0.5, len(rows)))
n_rows = len(data)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.2, bottom=0.2)

plt.ylabel("Loss in ${0}'s".format(value_increment))
plt.yticks(values * value_increment, ['%d' % val for val in values])
plt.xticks([])
plt.title('Loss by Disaster')

plt.show()